pybaseball github with install instructions: https://github.com/jldbc/pybaseball <br>
Documentation for pybaseball library: https://github.com/jldbc/pybaseball/tree/master/docs <br>
Statcast header meanings: https://baseballsavant.mlb.com/csv-docs

In [1]:
import pybaseball
import random
import itertools
# import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import date, timedelta, datetime

import warnings
warnings.filterwarnings("ignore")

In [12]:
teams = {
    'NYY' : 'New York Yankees',
    'LAD' : 'Los Angeles Dodgers',
    'NYM' : 'New York Mets',
    'TOR' : 'Toronto Blue Jays',
    'ATL' : 'Atlanta Braves',
    'MIN' : 'Minnesota Twins',
    'PHI' : 'Philadelphia Phillies',
    'SFG' : 'San Francisco Giants',
    'BOS' : 'Boston Red Sox',
    'COL' : 'Colorado Rockies',
    'CHC' : 'Chicago Cubs',
    'STL' : 'St. Louis Cardinals',
    'HOU' : 'Houston Astros',
    'MIA' : 'Miami Marlins',
    'SEA' : 'Seattle Mariners',
    'LAA' : 'Los Angeles Angels',
    'MIL' : 'Milwaukee Brewers',
    'WSN' : 'Washington Nationals',
    'CLE' : 'Cleveland Guardians',
    'CIN' : 'Cincinnati Reds',
    'ARI' : 'Arizona Diamondbacks',
    'SDP' : 'San Diego Padres',
    'TBR' : 'Tampa Bay Rays',
    'TEX' : 'Texas Rangers',
    'BAL' : 'Baltimore Orioles',
    'KCR' : 'Kansas City Royals',
    'PIT' : 'Pittsburgh Pirates',
    'CHW' : 'Chicago White Sox',
    'DET' : 'Detroit Tigers',
    'OAK' : 'Oakland Athletics'
    
}
standings = pybaseball.standings(2022)
season_standings = standings[0]
for x in range(len(standings)-1):
    season_standings = season_standings.append(standings[x+1]).reset_index(drop = True) 

In [ ]:
def simulation(year, home_team, away_team):
    home_data = pybaseball.schedule_and_record(2022, home_team)
    away_data = pybaseball.schedule_and_record(2022, away_team)
    model_data = home_data[['R', 'RA']]
    home_run_scored = home_data['R'].dropna()
    home_run_allowed = home_data['RA'].dropna()
    home_avg_scored = np.average(home_run_scored)
    home_avg_allowed = np.average(home_run_allowed)
    home_adj_scored = np.sqrt(home_avg_scored*home_avg_allowed) + 0.32 #Home field advantage adjustment
    home_run_scored_std = home_run_scored.std()
    
    model_data = away_data[['R', 'RA']]
    away_run_scored = away_data['R'].dropna()
    away_run_allowed = away_data['RA'].dropna()
    away_avg_scored = np.average(away_run_scored)
    away_avg_allowed = np.average(away_run_allowed)
    away_adj_scored = np.sqrt(away_avg_scored*away_avg_allowed)
    away_run_scored_std = away_run_scored.std()
    
    home_team_record = season_standings.loc[season_standings['Tm'] == teams[home_team]].reset_index(drop=True)
    home_team_win = round((int(home_team_record['W'][0]) / (int(home_team_record['W'][0]) + int(home_team_record['L'][0]))) * 100,1)
    
    away_team_record = season_standings.loc[season_standings['Tm'] == teams[away_team]].reset_index(drop=True)
    away_team_win = round((int(away_team_record['W'][0]) / (int(away_team_record['W'][0]) + int(away_team_record['L'][0]))) * 100,1)
    
    WP_home = home_team_win * (1-away_team_win) / ((home_team_win * (1-away_team_win)) + (away_team_win * (1-home_team_win)))
    WP_away = away_team_win * (1-home_team_win) / ((away_team_win * (1-home_team_win)) + (home_team_win * (1-away_team_win)))
    
    WP_home = round(WP_home*100,1)
    WP_away = round(WP_away*100,1)
    
    home = 0
    away = 0
    for x in range(10000):
        home_score = (norm.ppf(random.uniform(0, 1),home_adj_scored,home_run_scored_std) * .5) + (WP_home *.5)
        
        away_score = (norm.ppf(random.uniform(0, 1),away_adj_scored,away_run_scored_std) * .5) + (WP_away *.5)
        if home_score>away_score:
            home+=1
        else:
            away+=1
    print(home_team + ':' + str(round(home/10000*100,1)) + '% '+ away_team + ':' + str(round(away/10000*100,1)) + '%')

In [ ]:
day_of_week = datetime.today().strftime('%A')

today = date.today()
d4 = today.strftime("%b-%d-%Y")
d4 = d4.replace('-', ' ')
today = d4.split(' ')
today.pop()
if today[1][0] == '0':
    today[1] = today[1][1]
new_date = day_of_week + ', ' + today[0] + ' ' + today[1]

In [ ]:
team_abr = list(teams)
games = []
for x in team_abr:
    test = pybaseball.schedule_and_record(2022, x)
    test = test.loc[test['Date'] == new_date].reset_index(drop=True)
    if len(test) > 0:
        home_away = test['Home_Away'][0]
        if home_away == '@':
            games.append([test['Opp'][0], x])
        else:
            games.append([x,test['Opp'][0]])
# games
games.sort()
todays_games = list(games for games,_ in itertools.groupby(games))

In [ ]:
for x in todays_games:
    simulation(2022, x[0], x[1])

In [10]:
day_of_week_yesterday = datetime.today() - timedelta(days=1)
day_of_week_yesterday = day_of_week.strftime('%A')

yesterday = date.today() - timedelta(days=1)
d4 = yesterday.strftime("%b-%d-%Y")
d4 = d4.replace('-', ' ')
yesterday = d4.split(' ')
yesterday.pop()
if yesterday[1][0] == '0':
    yesterday[1] = yesterday[1][1]
yesterday_date = day_of_week_yesterday + ', ' + yesterday[0] + ' ' + yesterday[1]
yesterday_date

'Thursday, Jun 23'

In [25]:
team_abr = list(teams)
games = []
for x in team_abr:
    test = pybaseball.schedule_and_record(2022, x)
    test = test.loc[test['Date'] == yesterday_date].reset_index(drop=True)
    if len(test) > 0:
        home_away = test['Home_Away'][0]
        if home_away == '@':
            games.append([test['Opp'][0], x])
        else:
            games.append([x,test['Opp'][0], test['W/L'][0]])
# games
games.sort()
yesterday_games = list(games for games,_ in itertools.groupby(games))
results = []
for x in range(len(yesterday_games)):
    if len(yesterday_games[x]) == 3:
        results.append(yesterday_games[x])
results

[['ATL', 'SFG', 'W'],
 ['CHW', 'BAL', 'L'],
 ['CIN', 'LAD', 'L'],
 ['MIA', 'COL', 'W'],
 ['MIL', 'STL', 'W'],
 ['MIN', 'CLE', 'W'],
 ['NYY', 'HOU', 'W-wo'],
 ['OAK', 'SEA', 'L'],
 ['PIT', 'CHC', 'W-wo'],
 ['SDP', 'PHI', 'L']]